In [1]:
from sklearn.datasets import fetch_california_housing
import numpy as np
import tensorflow as tf

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error


In [3]:
house = fetch_california_housing()
x = house.data
y = house.target

In [0]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)


scaler2 = StandardScaler()
scaler2.fit(y.reshape(-1,1))
y = scaler2.transform(y.reshape(-1,1))

In [6]:
print(x.shape,y.shape)

(20640, 8) (20640, 1)


In [0]:
m,n = x.shape
x_bias = np.c_[np.ones((m,1)),x]
X_train,X_test,y_train,y_test = train_test_split(x_bias,y,test_size = 0.20,random_state = 105)
m,n = X_train.shape

In [9]:
print(X_train.shape,X_test.shape)

(16512, 9) (4128, 9)


In [0]:
tf.reset_default_graph()

In [11]:
W = tf.get_variable(name = "W",shape=[n, 1],initializer=tf.contrib.layers.xavier_initializer(),dtype = tf.float64)
lr =  1
output = tf.matmul(X_train,W,name = 'output')
diff = tf.subtract(output,y_train,name = 'diff')
loss = tf.reduce_mean((1/ (2 * m)) * tf.square(diff),name = 'loss')

grads1 = tf.gradients(loss,[W])[0]
grads2 = tf.linalg.pinv(tf.matmul(X_train.T,X_train)/m)
second_term = tf.matmul(grads2,grads1)
ass = tf.assign(W , W - (lr * second_term))

op = tf.global_variables_initializer()
diff = 9999

with tf.Session() as sess:
  sess.run(op)
  while(diff > 0.0001):
    
    W1 = W.eval()
    W1 = np.copy(W1)
    
    sess.run(ass)
    
    W2 = W.eval()
    W2 = np.copy(W2)
    
    diff = np.sum(np.abs(W2-W1))
    
    print(diff)  
  good_W = W.eval()
  
  

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

0.00030752245898453545
0.00030750383480454557
0.0003074852117524868
0.0003074665898284007
0.00030744796903207916
0.0003074293493633834
0.000307410730822244
0.0003073921134089941
0.00030737349712280093
0.00030735488196442784
0.00030733626793334745
0.0003073176550294765
0.0003072990432530093
0.00030728043260334903
0.000307261823081148
0.0003072432146858095
0.00030722460741754165
0.00030720600127602526
0.00030718739626151015
0.0003071687923737604
0.00030715018961229024
0.00030713158797790463
0.00030711298746979865
0.00030709438808844414
0.00030707578983335537
0.00030705719270458787
0.0003070385967020861
0.0003070200018258362
0.0003

In [0]:
mae_train = mean_absolute_error(y_train,np.dot(good_W.T,X_train.T).T)
mae_test = mean_absolute_error(y_test,np.dot(good_W.T,X_test.T).T)

In [13]:
print(mae_train,mae_test)

(0.5775548382762176, 0.5723903726551992)

In [0]:
tf.reset_default_graph()